- Import the necessary libraries and load data.

In [2]:
import pandas as pd
from utils.pre_processing_functions import encode_sub_type, convert_build_condition, encode_kitchen, get_latitude, get_longitude, get_distance

df = pd.read_csv('../immo_eliza_analysis/cleaned-data.csv')

We could now search for and delete the duplicates.
However, after closer examination and cross validation for a couple of the duplicates,
it has been discovered that some of the "duplicates" are properties that are part of real state projects, 
thus having similar information.

It is important to notice that after the dataset is further processed and some features are removed,
more duplicates might (or not) arise. Therefore, duplicates will be evaluated again at a later stage.

- Check for missing data: <br>
Since the data set was cleaned for the analysis part, it now contains no missing values.

In [3]:
df.isna().sum()

zip_code                      0
commune                       0
province                      0
type_of_property              0
subtype_of_property           0
price                         0
building_condition            0
facade_number                 0
living_area                   0
equipped_kitchen              0
bedroom_nr                    0
swimming_pool                 0
furnished                     0
open_fire                     0
terrace                       0
garden                        0
plot_surface                  0
sub_property_group_encoded    0
dtype: int64

- Drop unecessary colums: 
    - swimming_pool, furnished, open_fire, sub_property_group_encoded <br>
- Drop columns which have high correlation with others: 
    - bedroom_nr


In [4]:
df.drop(['bedroom_nr', 'swimming_pool', 'furnished', 'open_fire', 'sub_property_group_encoded'], axis=1, inplace=True)

- Drop observations under "other" type of property.

In [5]:
other_properties = df[df['subtype_of_property'].isin(['other property', 'mixed use building', ])].index
df.drop(other_properties, inplace=True)

In [7]:
# Make dictionary to ecode property subtype
sub_types_dict = {'kot': 0,
'chalet': 1,
'flat studio' : 2,
'service flat': 3,
'bungalow': 4,
'town house': 5,
'ground floor': 6,
'apartment': 7,
'house': 8,
'triplex': 9,
'farmhouse': 10,
'loft': 11,
'duplex': 12,
'apartment block': 13,
'country cottage': 14,
'penthouse': 15,
'mansion': 16,
'villa': 17,
'exceptional property': 18,
'manor house': 19,
'castle': 20}

In [8]:
# Apply encode subtype function and convert values to int
df['subtype_ecoded'] = df['subtype_of_property'].apply(lambda x: encode_sub_type(x, sub_types_dict))
df['subtype_ecoded'] = df['subtype_ecoded'].astype(int)

- Converting 'building condition' to numeric values. <br>

Removed observations with 'no info' about the building condition. <br>
Although some of those observations correspond to new projects, there is no way to confirm if all 'no info' properties are 'new'.

In [9]:
# Drop unknown building condition rows:
unknown_building_state = df[df['building_condition'] == 'no info'].index
df.drop(unknown_building_state, inplace=True)

# Apply function to convert building condition to numberic
df['building_condition'] = df['building_condition'].apply(convert_build_condition)

Further preprocessing our data, we discover that there are two appartments with 5 facades. They corresponds to a project (immoweb id 20147859) of apartments of 1,2 and 3 rooms, not a specific one with 5 facades, for which it was decided to remove this observation.

In [10]:
# Remove apartments with 5 facades
drop_by_facades = df[(df['facade_number'] > 4) & (df['type_of_property'] == 0)].index
df.drop(drop_by_facades, inplace=True)

- 'equipped_kitchen' still needs to be changed to a numerical value: {not installed: 0, installed: 1} 

In [11]:
print(df['equipped_kitchen'].value_counts())

# change it to numerical values
df['equipped_kitchen'] = df['equipped_kitchen'].apply(encode_kitchen)

# check the change was made
print('Checking that the change was made:')
df['equipped_kitchen'].value_counts()

equipped_kitchen
equipped         6767
not installed    6700
installed        5640
Name: count, dtype: int64
Checking that the change was made:


equipped_kitchen
2    6767
0    6700
1    5640
Name: count, dtype: int64

Now, there are nearly 900 houses without plot size, which might affect the model's predictions:

In [ ]:
# drop houses without plot size
zero_plot_surface = df[(df['type_of_property'] == 1) & (df['plot_surface'] == 0)]
print(f'Houses without plot surfacce: {zero_plot_surface.shape[0]}')

Houses without plot surfacce: 893


Since it represents about 10% of the total observations for houses, it was decided to add the mean plot size for houses in its commune to these observations. <br><br>
However, when computing the mean plot size per commune, it was discovered that some properties have very small values, such as 50 (sqm) or less, <br> which seemed very unlikely:

In [13]:
houses_df = df[df['type_of_property'] == 1]
small_plot_houses = houses_df[houses_df['plot_surface'] <= 50].shape[0]
print(f'Houses with plot surface < 50: {small_plot_houses}')

super_small_house_plots = houses_df[houses_df['plot_surface'] < 25].shape[0]
print(f'Houses with plot surface < 25: {super_small_house_plots}')

Houses with plot surface < 50: 1016
Houses with plot surface < 25: 906


For this, it was decided to also drop the 'plot_surface' column.

In [14]:
df.drop('plot_surface', axis=1, inplace=True)

- Check for duplicates <br>
As per the below data frame, these represent no duplicates. Therefore, they will be kept in the data set.

In [15]:
df[df.duplicated(['commune', 'price', 'living_area'])].sort_values(by= 'commune').head()

,zip_code,commune,province,type_of_property,subtype_of_property,price,building_condition,facade_number,living_area,equipped_kitchen,terrace,garden,subtype_ecoded
11589,9300,Aalst,Oost-Vlaanderen,0,apartment,260250.0,2,2.0,110,2,8,0,7
1858,9300,Aalst,Oost-Vlaanderen,0,penthouse,549000.0,2,2.0,180,1,15,0,15
22722,9300,Aalst,Oost-Vlaanderen,1,house,1350000.0,2,2.0,992,1,0,0,8
21053,9300,Aalst,Oost-Vlaanderen,1,house,239000.0,2,2.0,120,1,1,0,8
20238,8211,Aartrijke,West-Vlaanderen,1,house,338500.0,2,3.0,159,1,1,0,8


### Feature engineering <br>
Since the data set contains the zip code where the property is located and the commune it belogs to, the distance to the province's capital was added.<br><br>

For this, we first need to import the zipcodes and its corresponding coordinates.

In [16]:
zip_codes_df = pd.read_csv('utils/zipcode_belgium.csv')

zip_codes_df.index = zip_codes_df.index + 1  # Shift all other rows down
zip_codes_df.loc[0] = zip_codes_df.columns # Add info for Brussels, beacuase it is incorrectly used at the column names in the original csv file.

# Correct column names
column_names = ['zip_code_col', 'commune', 'longitude', 'latitude']
zip_codes_df = zip_codes_df.sort_index()
zip_codes_df.columns = column_names

# Correct data type of the columns, because it caused error to function
zip_codes_df['zip_code_col'] = zip_codes_df['zip_code_col'].astype(int)
zip_codes_df['latitude'] = zip_codes_df['latitude'].astype(float)
zip_codes_df['longitude'] = zip_codes_df['longitude'].astype(float)

In [17]:
# Add latitude and logitude to all observations
df['latitude'] = df['zip_code'].apply(lambda x: get_latitude(x, zip_codes_df))
df['longitude'] = df['zip_code'].apply(lambda x: get_longitude(x, zip_codes_df))

Now, we assign the corresponding distance to each property. <br>
First, we create a data frame with the latitude and longitude of each province's capital:

In [18]:
provinces = [
['West-Vlaanderen', 'Oost-Vlaanderen', 'Antwerpen', 'Liège', 'Vlaams Brabant', 'Hainaut', 'Brabant Wallon', 'Namur', 'Luxembourg', 'Limburg', 'Bruxelles'],
[51.2085, 51.05, 51.2199, 50.6402, 50.8791, 60.3913, 50.7154, 50.4649, 49.6116, 50.9305, 50.8477],
[3.2251, 3.7304, 4.415, 5.5689, 4.7025, 5.3221, 4.6177, 4.865, 6.1319, 5.3324, 4.3572]
 ]    
col_names = ['province', 'latitude', 'longitude']
provinces_df = pd.DataFrame(provinces).T
provinces_df.columns = col_names
provinces_df['latitude'] = provinces_df['latitude'].astype(float)
provinces_df['longitude'] = provinces_df['longitude'].astype(float)

Now, the distance of each property to its province's capilat is calculated and added:

In [19]:
df['km_to_capital'] = df.apply(lambda row: get_distance(row, provinces_df), axis=1)

Finally, we can drop the columns we no longer need: <br> 
-sub_type_of_property, zip_code, commune and provice columns.

In [20]:
df.drop(['subtype_of_property', 'zip_code', 'commune', 'province', 'latitude', 'longitude'], axis=1, inplace=True)

In [21]:
df.head()

,type_of_property,price,building_condition,facade_number,living_area,equipped_kitchen,terrace,garden,subtype_ecoded,km_to_capital
0,0,149000.0,2,2.0,48,1,9,0,7,3.33
1,0,248000.0,2,2.0,91,1,2,0,7,3.91
2,0,229000.0,2,2.0,100,0,26,0,7,6.62
3,0,470000.0,2,2.0,179,2,10,0,7,5.29
4,0,480000.0,2,2.0,116,1,7,0,7,1.07


In [22]:
df.dtypes

type_of_property        int64
price                 float64
building_condition      int64
facade_number         float64
living_area             int64
equipped_kitchen        int64
terrace                 int64
garden                  int64
subtype_ecoded          int64
km_to_capital         float64
dtype: object

The data is ready to be used in machine learning.

In [23]:
df.to_csv('ml_data.csv', index=False)